
# Opening Range — Spec & Config (Read‑Only)

> **Important:** This notebook is **documentation-only**. **Do not edit parameters here.**  
> The single source of truth lives in **`config/strategy.yml`** and **`config/instruments.yml`**.  
> All analysis notebooks will **load** those YAML files at the top.



## Strategy (Human Summary)

- **Instrument:** NSXUSD (1-minute bars)  
- **Opening Range (OR):** 09:30–10:00 **inclusive** (America/New_York)  
- **Entry time:** **10:22**  
- **Hard exit:** **12:00**  
- **Zones:** Long if price in **top 35%** of OR; Short if in **bottom 35%**; else **no trade**  
- **Risk:** SL **25 points**, TP **75 points**, **$80 per point**  
- **Execution:** One trade per day; fills at level on intrabar high/low; round to tick (TBD)  
- **Policy:** Skip days with missing OR/10:22/12:00 or gaps; no premarket present

## Opening Range Strategy — Quick Reference

**Instrument:** NSXUSD (CFD-style NASDAQ-100 cash index, quoted nearly 24h; we slice to NY cash session)  
**Timezone:** America/New_York  
**Data cadence:** 1-minute OHLC (volume ignored)

### Session & Windows
- **Opening Range (OR):** **09:30 → 10:00 (inclusive)** ⇒ 31 bars.
- **Decision time:** **10:22** (one decision per day).
- **Hard exit:** **12:00** (close any open trade at market).

### Parameters (baseline)
| Setting | Value |
|---|---|
| Top / Bottom zone | 35% / 35% of OR |
| Stop Loss | **25** points |
| Take Profit | **75** points |
| Max trades/day | 1 |
| $ per point | **$80** (reporting) |
| Slippage/fees | Off in baseline (configurable later) |

### OR definitions
- `OR_high` = max(**high**) from 09:30–10:00 (inclusive)  
- `OR_low`  = min(**low**)  from 09:30–10:00 (inclusive)  
- `R` = `OR_high − OR_low` (points)

**Zone cutoffs**
- **Bottom-35% cutoff:** `OR_low + 0.35 * R`  
- **Top-35% cutoff:**    `OR_high − 0.35 * R`

### Entry decision at 10:22
Let `P_10:22` = **close** of the 10:22 bar:
- If `P_10:22 > Top-35% cutoff` ⇒ **Long**
- If `P_10:22 < Bottom-35% cutoff` ⇒ **Short**
- Else ⇒ **No trade**

### Risk management & exits
Given entry price `E` at 10:22:
- **Long:** `SL = E − 25`, `TP = E + 75`  
- **Short:** `SL = E + 25`, `TP = E − 75`
- Walk minute-by-minute **10:22 → 12:00**:
  1) Exit on **TP** touch,
  2) Else exit on **SL** touch,
  3) Else exit at **12:00** (market).
- **P&L (points):**  
  Long = `exit − E` ; Short = `E − exit`  
  **P&L ($)** = `P&L (pts) × $80` (later we can subtract fees/slippage).

> **Note:** Some data feeds only store bar **close** (your baseline code checked closes). A more realistic variant detects barrier touches with bar **high/low**; if both SL & TP could be hit in the same bar, choose a documented tie-break (e.g., conservative “SL before TP”). We’ll keep this configurable later.

### Valid-day requirements (data audit)
A day is tradable only if **all** hold:
- OR window has **31 bars**; `OR_range > 0`.
- Trade window **09:30–12:00** has **no missing/duplicate minutes**.
- Bars at **10:22** and **12:00** exist.

---

### Worked example (2020-01-03, 09:30–10:00)
From the provided slice:
- `OR_high = 8822.17` (at 09:44)  
- `OR_low  = 8752.10` (at 09:30)  
- `R = 70.07`  
- **Bottom-35% cutoff** = `8752.10 + 0.35×70.07 = 8776.6245`  
- **Top-35% cutoff**    = `8822.17 − 0.35×70.07 = 8797.6455`  
Decision at **10:22** uses the 10:22 **close** relative to these cutoffs.




## Why configs live in YAML
- Reproducible and auditable (versioned in Git)  
- Shareable across notebooks/scripts without hidden state  
- Easy to run sensitivity tests by swapping config files



## Next
- Confirm **tick_size**, **fees_per_side_usd**, and **slippage_points** in `config/instruments.yml`.  
- Start `02_data_audit.ipynb`: it will load these configs and verify the raw data coverage (09:30–12:00), OR completeness, and presence of 10:22 and 12:00.


In [5]:

# Read-only viewer for the YAML configs (optional to run later)
# This cell is safe to run; it does not change any parameters.
from pathlib import Path
print("Config files located at:", (Path.cwd().parent / "config"))

try:
    import yaml
    cfg_dir = Path("../config")
    with open(cfg_dir / "strategy.yml", "r", encoding="utf-8") as f:
        strategy = yaml.safe_load(f)
    with open(cfg_dir / "instruments.yml", "r", encoding="utf-8") as f:
        instruments = yaml.safe_load(f)

    print("\n--- strategy.yml ---")
    print(yaml.dump(strategy, sort_keys=False))
    print("\n--- instruments.yml ---")
    print(yaml.dump(instruments, sort_keys=False))

except Exception as e:
    print("If PyYAML is not installed yet, install it in your virtualenv:")
    print("  pip install pyyaml")
    print("Then re-run this cell.")
    print("\nRaw file preview:")
    for name in ["strategy.yml", "instruments.yml"]:
        p = Path("../config") / name
        print(f"\n--- {name} ---")
        print(p.read_text(encoding="utf-8"))


Config files located at: d:\Projects\OpeningRange\config
If PyYAML is not installed yet, install it in your virtualenv:
  pip install pyyaml
Then re-run this cell.

Raw file preview:

--- strategy.yml ---
strategy:
  name: "Opening Range — NSXUSD"
  version: 1

parameters:
  zones:
    top_pct: 0.35 # long if price in top 35% of OR
    bottom_pct: 0.35 # short if price in bottom 35% of OR
  risk:
    stop_loss_points: 25
    take_profit_points: 75
  execution:
    one_trade_per_day: true
    allow_reentry_same_day: false
    fill_detection: "intrabar_high_low" # detect SL/TP with bar high/low
    fill_price_policy: "at_level" # fill exactly at SL/TP level
    round_to_tick: true
  capital:
    initial_usd: 100000

evaluation:
  metrics:
    - net_return
    - max_drawdown
    - win_rate
    - profit_factor
    - expectancy
    - avg_win
    - avg_loss
    - cagr
    - volatility
    - sortino
  reporting:
    save_trade_log: true
    save_equity_curve: true


--- instruments.yml ---
ma